In [11]:
##tentative implémentation code Thibault

In [64]:
from collections import defaultdict
from collections import namedtuple
from collections import Counter
from tqdm import tqdm

from thibaultScript import (
    import_known_tokens,
    compile_scores,
    convert_raw#,
#    vjui
)

from pandas import DataFrame, Series
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline


In [65]:
KNOWN_TOKENS, KNOWN_LEMMAS = import_known_tokens()

In [67]:
#KNOWN_TOKENS

In [68]:
#TASKS = "lemma,Deg,Numb,Person,Mood_Tense_Voice,Case,Gend,pos".split(",")
TASKS = "lemma,POS".split(",")

In [69]:
Texts = []
GOLD = []

Milestones = set(list(range(0, 7270)))
nb_tokens = 0

def filter_gold(data):
    return [lst for lst in data if lst]


import regex

punkts = regex.compile(r"\W+")
greek = regex.compile(r"\p{Greek}+")

def keep_tokens(token):
    if greek.match(token):
        print(token)
        return False
    return True


with open("nca-sample-naomicorr.tsv") as f:
    header = []
    current_text = None
    for lineno, line in enumerate(f):
        line = line.strip().split("\t")
        if lineno == 0:
            header = line
            continue
        
        line = dict(zip(header, line))
        
        if line["form"].startswith("[REF:"):
            GOLD = filter_gold(GOLD)
            Texts.append((line["form"], nb_tokens, len(GOLD)))
            GOLD.append([])
            continue
        
        if line["POS"] == "PONfrt":
            if GOLD[-1] != []:
                GOLD = filter_gold(GOLD)
                GOLD.append([])
        elif keep_tokens(line["form"]):
            GOLD[-1].append(line)
            nb_tokens += 1
            

if GOLD[-1] == []:
    GOLD = GOLD[:-1]
print(f"{len(Texts)} texts found")
print(f"{len(GOLD)} sentences found")
TextsLengths = {
    
}
# Compute texts Lengths
for cur, nxt in zip(Texts, Texts[1:] + [None]):
    if nxt is None:
        TextsLengths[cur[0]] = nb_tokens - cur[1]
    else:
        TextsLengths[cur[0]] = nxt[1] - cur[1]
# Update titles
#TextsTitles = {
#    urn: f"{title} ({TextsLengths.get(urn, '?')} mots)"
#    for urn, title in TextsTitles.items()
#}
#print(TextsTitles)

278 texts found
1677 sentences found


In [73]:
TextsTitles = [t[5:-1] for t in TextsLengths.keys()]

In [74]:
TextsTitles

['ChastVergiS_A_69',
 'PsLorrA_672',
 'PerNeslesTabJ_602',
 'MerlinP_956',
 'MerlinP_1603',
 'CharroiSch_C*_1223',
 'CharroiSch_C*_1250',
 'ElesB_G_140',
 'PercLo_Q_58',
 'NoomenFabl_transB*_733',
 'MarieFab_K_903',
 'MarieFab_K_2052',
 'ContPerc4Lo_A_1262',
 'ContPerc4Lo_A_1401',
 'Turpin1A_578',
 'MarieMil_S_361',
 'MarieEquitS_H_267',
 'RenN_I_3',
 'CassidP_227',
 'MaccabES_185',
 'BesantR_199',
 'RoseLLec_1147',
 'RoseLLec_2516',
 'RoseLLec_2874',
 'RoseLLec_3556',
 'RenN_K_385',
 'PercLo_U_958',
 'PercLo_U_1215',
 'FillePonth1B2_94',
 'PeanGatS1_2_547',
 'PeanGatS1_2_1340',
 'PeanGatS1_2_1668',
 'RobOmMirL_172',
 'RobOmMirL_503',
 'FetRomF1_290',
 'FetRomF1_417',
 'ComtePoitM_542',
 'MerlinM_236',
 'MerlinM_3036',
 'MarieFab_C_2872',
 'AthisH_2252',
 'AthisH_2364',
 'AthisH_3340',
 'EvEnfB_50',
 'ClefD_1108',
 'ClefD_1426',
 'ClefD_2007',
 'ClefD_2180',
 'Aucun_201',
 'Aucun_206',
 'CartTournaiF*_904',
 'CartTournaiF*_2491',
 'SGregA1S_2348',
 'SGregA1S_2525',
 'PeanGatS1_3_1950',

In [75]:
PLATINUM = GOLD #convert_raw(GOLD, task_list=TASKS, lemma_fn=lambda x:x, form_fn=lambda x:x, 
                #       pos_fn=lambda x:x, clitics_starts_with_accollade=True,
                #      clitics_are_duplicate=True)

In [78]:
TOKENS = [
    [x["form"] for x in sentence]
    for sentence in PLATINUM
]
TOKENS[0]

['Longue',
 'atente',
 'vous',
 'poroit',
 'nuire',
 ',',
 'ce',
 'm',
 '’',
 'est',
 'vis',
 ':',
 'si',
 'lo',
 'que',
 'vous',
 'soiiés',
 'amis',
 'en',
 'un',
 'haut',
 'liu',
 ',',
 'se',
 'vous',
 'veés',
 'que',
 'vous',
 'i',
 'soiiés',
 'bien',
 'amés']

In [79]:
TASKS

['lemma', 'POS']

In [80]:
from pie.tagger import Tagger
from pie.utils import chunks
DEVICE = "cpu"
BATCH_SIZE = 128


OUTPUT = []
TEST_NEW = True

#tagger = Tagger(device="cuda", batch_size=100, lower=False, tokenize=False)
tagger = Tagger(device=DEVICE, batch_size=BATCH_SIZE, lower=False, tokenize=False)
if TEST_NEW:
    for task in TASKS:
        tagger.add_model(f"modeles/{task}.tar", task)
#else:
#    tagger.add_model("../../../latin-lasla-models/lasla-plus.tar", *TASKS)


for chunk in tqdm(chunks([(sent, len(sent)) for sent in TOKENS], tagger.batch_size)):
    tagged, tasks = tagger.tag(*zip(*chunk))#, use_beam=True)
    OUTPUT.extend([
        [
            (token, dict(zip(tasks, result)))
            for token, result in sentence
        ]
        for sentence in tagged
    ])


14it [01:25,  6.13s/it]


In [82]:
from collections import defaultdict, Counter


RESULTS, RAW_SCORES, RAW_SCORES_NOT_EMPTY, ERRORS, SCORES_KNOWN = compile_scores(
    OUTPUT, PLATINUM, task_list=TASKS, known_tokens=KNOWN_TOKENS, known_lemmas=KNOWN_LEMMAS)

In [83]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tabulate import tabulate
from IPython.display import HTML, display

data = [["Task", "Accuracy", "Accuracy on V != _"]]


for task, (pred, truth) in RAW_SCORES.items():
    (pred_limited, truth_limited) = RAW_SCORES_NOT_EMPTY[task]
    data.append([
        task,
        "{0:.2f}".format(accuracy_score(truth, pred)*100),
        "{0:.2f}".format(accuracy_score(pred_limited, truth_limited)*100)
    ])
    
df = DataFrame([x[1:] for x in data[1:]], columns=data[0][1:], index=[x[0] for x in data[1:]]).sort_index()
display(HTML(df.to_html()))
#print(df.to_latex())



,Accuracy,Accuracy on V != _
POS,89.78,89.78
lemma,88.08,88.08


## ACCURACY PAR TEXTE PAS INTÉRESSANTE ICI: textes trop courts
MilestonesSentences = [
    (tid, begin, end)
    for (tid, _, begin), (_, _, end) in zip(Texts, Texts[1:]+[(None, None, None)])
] # Those are sentences IDs

def count_tokens(begin, end):
    return len([tok for sent in PLATINUM[begin:end] for tok in sent])

MILESTONES = [count_tokens(beg, end) for _, beg, end in MilestonesSentences]
MILESTONES = [sum(MILESTONES[:index]) + value for index, value in enumerate(MILESTONES)]

SPLITS_TASK = {
    task: []
    for task in RAW_SCORES
}
SIZES = []


lengths = []
support_done = False
for task, (pred, truth) in RAW_SCORES.items():
    ms_start = 0
    for milestone in MILESTONES:
        SPLITS_TASK[task].append(accuracy_score(truth[ms_start:milestone], pred[ms_start:milestone]))
        length = milestone - ms_start + 1
        if not support_done:
            SIZES.append(length)
        ms_start = milestone
    support_done = True
    

MilestonesSentences[1][0]
#MILESTONES
#SIZES
#TextsTitles[tid]
#TextsTitles



sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})

DF_Tardif = DataFrame(SPLITS_TASK, index=[
    #"{} ({} mots)".format(TextsTitles[tid], length) for (tid, *_), length in zip(MilestonesSentences, SIZES)
    "{} ({} mots)".format(TextsTitles[0], length) for (*_), length in zip(MilestonesSentences, SIZES)
])

plt.figure(figsize=(10,10))
cmap = sns.cubehelix_palette(250, hue=0.05, rot=0, light=1, dark=0, as_cmap=True)
ax = sns.heatmap(DF_Tardif, annot=True, cmap=cmap)



In [89]:
from collections import defaultdict

# Dict[Text_id, Dict[POS, [0, 1]]] where 0 = false, 1 = true
Stats_Per_Pos = defaultdict(list)
_, Scores_all2, *_, Score_Knowns = compile_scores(OUTPUT, PLATINUM, 
                                                     task_list=TASKS, known_tokens=KNOWN_TOKENS, 
                                                     known_lemmas=KNOWN_LEMMAS)

NBTOKENS = 0
NBERRORS = 0

for (lemma_pred, lemma_truth,  pos_truth) in zip(
   Scores_all2["lemma"][0],
   Scores_all2["lemma"][1],
   Scores_all2["POS"][1]
):
    Stats_Per_Pos[pos_truth].append(int(lemma_pred == lemma_truth))
    NBTOKENS +=1
    NBERRORS += int(lemma_pred != lemma_truth)


In [97]:
LIMIT_TO_POS = {"VERcjg", "VERinf", "VERppe", "VERppa", "NOMcom", "ADJqua"}#LISTE À CONFIRMER


def compute_accuracy(trues, corpus_size, corpus_accuracy) -> int:
    return ((len(trues) - sum(trues)) / corpus_size) / corpus_accuracy

PosLemmaDetails = DataFrame([
    {"POS": pos, "Accuracy": sum(equals) / len(equals), "Impact": compute_accuracy(equals, NBTOKENS, NBERRORS/NBTOKENS)}
    for pos, equals in Stats_Per_Pos.items()
    #if pos in LIMIT_TO_POS
])

PosLemmaDetails

,POS,Accuracy,Impact
0,ADJqua,0.851590,0.041359
1,NOMcom,0.824211,0.201625
2,PROper,0.961419,0.024372
3,VERcjg,0.924239,0.086411
4,VERinf,0.915337,0.016987
5,PONfbl,0.820452,0.203594
6,PROdem,0.941504,0.005170
7,ADVgen,0.932874,0.035943
8,CONsub,0.929221,0.017233
9,PRE,0.979892,0.012802


In [60]:
##mes modifs
Impact_Acc_Per_Pos2 = Impact_Acc_Per_Pos.reset_index(level=0)
Impact_Acc_Per_Pos3 = Impact_Acc_Per_Pos2['index'].str.split('\n', expand=True)
Impact_Acc_Per_Pos3

,0,1
0,[REF:ChastVergiS_A_69],(Error Rate: 38.5%)
1,[REF:PsLorrA_672],(Error Rate: 30.9%)
2,[REF:PerNeslesTabJ_602],(Error Rate: 30.0%)
3,[REF:MerlinP_956],(Error Rate: 27.8%)
4,[REF:MerlinP_1603],(Error Rate: 26.1%)
...,...,...
57,[REF:IpH_603],(Error Rate: 30.8%)
58,[REF:IpH_3634],(Error Rate: 35.8%)
59,[REF:IpH_3829],(Error Rate: 31.9%)
60,[REF:IpH_5979],(Error Rate: 29.5%)


In [243]:
Impact_Acc_Per_Pos3["error"] = Impact_Acc_Per_Pos3[1].str.extract(r'\(Error Rate: (.*)%\)')
Impact_Acc_Per_Pos3.index = Impact_Acc_Per_Pos3[0].str.extract(r'\[REF:(.*)\]')
Impact_Acc_Per_Pos3
type(Impact_Acc_Per_Pos3)
#Impact_Acc_Per_Pos3["error"]

pandas.core.frame.DataFrame

In [244]:
Impact_Acc_Per_Pos3[Impact_Acc_Per_Pos3["error"]].astype(float)

KeyError: "None of [Index(['53.3', '38.2', '32.0', '44.3', '38.7', '37.3', '42.4', '42.6', '41.0',\n       '43.9', '40.4', '49.1', '47.4', '36.1', '39.6', '44.1', '30.3', '50.5',\n       '45.5', '40.0', '48.5', '39.2', '36.8', '47.1', '44.2', '45.7', '40.8',\n       '29.3', '36.6', '40.2', '32.7', '41.0', '40.3', '42.0', '47.0', '42.4',\n       '41.8', '40.3', '46.8', '41.9', '49.6', '40.4', '41.1', '46.5', '48.0',\n       '43.6', '42.9', '41.2', '41.2', '39.4', '42.1', '29.8', '46.0', '42.6',\n       '33.6', '42.2', '41.5', '39.4', '38.7', '46.9', '39.0', '44.2'],\n      dtype='object')] are in the [columns]"

In [239]:
Impact_Acc_Per_Pos4 = Impact_Acc_Per_Pos3["error"]
Impact_Acc_Per_Pos4
dfPlot = Impact_Acc_Per_Pos4[Impact_Acc_Per_Pos4.columns].astype(float)

AttributeError: 'Series' object has no attribute 'columns'

In [216]:
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
fig = plt.figure(figsize=(10,10))
cmap = sns.cubehelix_palette(250, hue=0.05, rot=0, light=1, dark=0, as_cmap=True)
ax = sns.heatmap(Impact_Acc_Per_Pos4, annot=True)#, cmap=cmap)
ax.set_title("Pourcentage d'erreurs occasionnées par POS")
fig.show()

IndexError: Inconsistent shape between the condition and the input (got (62, 1) and (62,))

<Figure size 720x720 with 0 Axes>

In [223]:
TextTitleSize[text_id].size
TextTitleSize[text_id].error_rate
Stats_Per_Pos.items

<function defaultdict.items>

In [237]:
ScatterPos = DataFrame([
    {
        "POS": pos,
        "Corpus": len(equals)/TextTitleSize[text_id].size, 
        "desErreurs": compute_accuracy(
            equals,
            TextTitleSize[text_id].size,
            TextTitleSize[text_id].error_rate
        )
    }
    for text_id, texts_values in Stats_Per_Pos.items()
        for pos, equals in texts_values.items()
        if pos in LIMIT_TO_POS
])

ScatterPos
#ScatterPos["Impact"] = ScatterPos["% des Erreurs"] / ScatterPos["% Corpus"] 

#texts_values.items
Stats_Per_Pos.items()

dict_items([(0, defaultdict(<class 'list'>, {'_': [1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1]})), (1, defaultdict(<class 'list'>, {'_': [1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1]})), (2, defaultdict(<class 'list'>, {'_': [1,

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,7), dpi=300)
# Draw plotbot
ScatterPos.boxplot(ax=ax1, column="Impact", by="POS")
ax1.set_title(label="")
ax1.axhline(y=1, label="Distribution normale", linestyle=":")
ax1.set_ylabel("Impact")
# ax1.text(y=0.8, x=5.6, s="Distribution\nnormale")
ax1.grid(True, color='b', linestyle='--', linewidth=0.2)

# Draw scatter
ax2 = sns.scatterplot(
    x="% Corpus", y="% des Erreurs", style="POS", 
    data=ScatterPos, alpha=0.8,
    color="g",
    ax=ax2
)
x = np.linspace(0, 0.4)
line = ax2.plot(x, x, linestyle=':', color="b", label="Distribution\nnormale")
# line_patch = mpatches.Patch(color='blue', linestyle="--", label="Distribution\nnormale")

handles, labels = ax2.get_legend_handles_labels()
ax2.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0, handles=handles)

fig.suptitle("")
fig.show()

nompro =  ScatterPos.loc[ScatterPos["POS"]=="NOMpro"]["Impact"]
print(nompro.median())#, nompro)
nomcom =  ScatterPos.loc[ScatterPos["POS"]=="NOMcom"]["Impact"]
print(nomcom.median())#, nomcom)
ver =  ScatterPos.loc[ScatterPos["POS"]=="VER"]["Impact"]
print(nomcom.median())#, nomcom)

FullDistrib = DataFrame([
    {
        "POS": pos,
        "% Corpus": len(equals)/TextTitleSize[text_id].size, 
        "% des Erreurs": compute_accuracy(
            equals,
            TextTitleSize[text_id].size,
            TextTitleSize[text_id].error_rate
        )
    }
    for text_id, texts_values in Stats_Per_Pos.items()
        for pos, equals in texts_values.items()
])
FullDistrib["Impact"] = ScatterPos["% des Erreurs"] / ScatterPos["% Corpus"]
FDGB = FullDistrib.groupby("POS").median()
print(FDGB)